# Import

In [1]:
import sys
import os
# 標準エラー出力を/dev/nullにリダイレクト
sys.stderr = open(os.devnull, 'w')

import pandas as pd
import numpy as np
import japanize_matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from geopy.distance import geodesic
import math as mt
import folium
import ast
from sklearn.preprocessing import StandardScaler

# データの読み込み

In [2]:
gad_df = pd.read_csv(r"C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\data\GAD_sapporo_Nov_weekend.csv")

In [3]:
df = gad_df.copy()
def grid3rd(lon, lat):  # 3次メッシュ(8桁) 8分割x10分割=80分割
    return (int(mt.floor(lat*120      / 80)) * 1000000 + int(mt.floor((lon-100))             ) * 10000 +  # 1次メッシュ
            int(mt.floor(lat*120 % 80 / 10)) * 1000    + int(mt.floor((lon-100)*80 % 80 / 10)) * 100 +    # 2次メッシュ
            int(mt.floor(lat*120 % 10))      * 10      + int(mt.floor((lon-100)*80)) % 10               ) 
mesho_list = []
for i in range(len(df)):
    mesh = grid3rd(df['start_lon'][i],df['start_lat'][i])
    mesho_list.append(mesh)
df['mesh_o'] = mesho_list

meshd_list = []
for i in range(len(df)):
    mesh = grid3rd(df['end_lon'][i],df['end_lat'][i])
    meshd_list.append(mesh)
df['mesh_d'] = meshd_list
gad_df = df.copy()

In [4]:
gad_df = gad_df.rename(columns={"end_activity_type":"purpose"})
gad_df = gad_df.rename(columns={"longest_distance_mode":"mode"})
gad_df['trav_time_timedelta'] = pd.to_timedelta(gad_df['trav_time'])
# timedeltaから分のみ（総分数）を取得
gad_df['trav_time_minutes'] = round(gad_df['trav_time_timedelta'].dt.total_seconds() / 60)
gad_df = gad_df.dropna(subset=['trav_time'])
gad_df = gad_df.drop(['dep_time_tz', 'arr_time_tz', 'next_dep_time_tz', 'wait_time', 'euclidean_distance', 'start_activity_type_id', 'end_activity_type_id', 'start_key_code', 'end_key_code', 'modes_id', 'work_lon', 'work_lat', '短縮名称'], axis=1)


In [5]:
gad_df['judge'] = (gad_df['mode'] == 'car') & (gad_df['trav_time_minutes'] <= 5) & (gad_df['traveled_distance'] <= 2000) | (gad_df['mode'] == 'walk') & (gad_df['traveled_distance'] >= 1200)

# 全体代替可能トリップチェーン抽出

In [6]:
gad_walk = gad_df[gad_df['mode'].isin(['walk'])].reset_index(drop=True)
gad_car = gad_df[gad_df['mode'].isin(['car'])].reset_index(drop=True)
short_car =  gad_car[(gad_car['trav_time_minutes'] <= 5) & (gad_car['traveled_distance'] <= 2000)].reset_index(drop=True)
long_walk = gad_walk[gad_walk['traveled_distance'] >= 1200].reset_index(drop=True)
agent_df = pd.concat([short_car, long_walk], ignore_index=True)

In [7]:
# 代替可能性があるトリップのユーザーID
id_list = list(set(agent_df['agent_id']))

In [9]:
def check_all_rows_valid(df):
    return ((df['judge'] == True) | ((df['judge'] == False) & (df['mode'] == 'walk'))).all()

potential_true_df = pd.DataFrame(columns=gad_df.columns)
potential_false_df = pd.DataFrame(columns=gad_df.columns)
for i in range(len(id_list)):
    df = gad_df[gad_df['agent_id'].isin([id_list[i]])]
    if check_all_rows_valid(df):
        potential_true_df = pd.concat([potential_true_df, df], ignore_index=True)
    else:
        potential_false_df = pd.concat([potential_false_df, df], ignore_index=True)  


In [13]:
potential_true_df.to_csv(r'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\data\休日全体代替可能data.csv')

# 分析 

In [ ]:
potential_df = pd.read_csv(r"C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\data\休日全体代替可能data.csv", index_col=0)

In [10]:
potential_df = potential_true_df

In [14]:
ture_df = gad_df[gad_df['judge'] == True].reset_index(drop=True)

## モード×目的のクロス集計表（トリップ単位）


In [21]:
purpose_list = list(true_df['purpose'].unique())
x_table = pd.DataFrame(columns=purpose_list,index = ['walk','car'])
y_table = pd.DataFrame(columns=purpose_list,index = purpose_list)

In [22]:
for i in range(len(purpose_list)):
    for j in ['walk','car']:
        x_table.loc[j, purpose_list[i]] = len(df[(df['purpose'] == purpose_list[i]) & (df['mode'] == j)])

In [23]:
for i in range(len(purpose_list)):
    for j in purpose_list:
        y_table.loc[j, purpose_list[i]] = len(df[(df['start_activity_type'] == purpose_list[i]) & (df['purpose'] == j)])

In [24]:
x_table

,others,eat,home,entertainment,retail,outdoor,grocery,medical,work
walk,31252,14493,26850,959,10852,1378,1806,1589,2222
car,164298,71738,214585,10980,80008,22584,18461,19278,8236


In [25]:
y_table

,others,eat,home,entertainment,retail,outdoor,grocery,medical,work
others,0,28518,112512,3510,30005,6277,5953,6316,2459
eat,26983,0,38406,1756,11850,1953,2223,2230,830
home,110532,42279,0,5395,45231,11851,10046,9320,6781
entertainment,3413,1529,4535,0,1400,372,279,307,104
retail,27326,11443,42217,1473,0,2462,2534,2585,820
outdoor,6416,2185,10941,372,2672,0,499,695,182
grocery,5354,2147,8922,263,2460,437,0,504,180
medical,5512,2061,9264,263,2529,529,512,0,197
work,1880,541,6814,65,660,182,122,194,0


## 移動手段×目的別のGIS

In [18]:
true_df = gad_df[gad_df['judge'] == True].reset_index(drop=True)

In [19]:
df = true_df.copy()
def grid3rd(lon, lat):  # 3次メッシュ(8桁) 8分割x10分割=80分割
    return (int(mt.floor(lat*120      / 80)) * 1000000 + int(mt.floor((lon-100))             ) * 10000 +  # 1次メッシュ
            int(mt.floor(lat*120 % 80 / 10)) * 1000    + int(mt.floor((lon-100)*80 % 80 / 10)) * 100 +    # 2次メッシュ
            int(mt.floor(lat*120 % 10))      * 10      + int(mt.floor((lon-100)*80)) % 10               ) 
mesho_list = []
for i in range(len(df)):
    mesh = grid3rd(df['start_lon'][i],df['start_lat'][i])
    mesho_list.append(mesh)
df['mesh_o'] = mesho_list

meshd_list = []
for i in range(len(df)):
    mesh = grid3rd(df['end_lon'][i],df['end_lat'][i])
    meshd_list.append(mesh)
df['mesh_d'] = meshd_list
true_df = df.copy()

KeyboardInterrupt: 

In [20]:
walk_true = true_df[true_df['mode'] == 'walk'].reset_index(drop=True)
car_true = true_df[true_df['mode'] == 'car'].reset_index(drop=True)

## カーネル密度推定

In [ ]:
true_df

,agent_id,dep_time,arr_time,activity_time,next_dep_time,trav_time,traveled_distance,mode,modes,start_activity_type,...,end_lat,longest_distance_mode_id,age,gender,home_lon,home_lat,geometry,trav_time_timedelta,trav_time_minutes,judge
0,0,12:30:00,12:33:43,01:52:17,14:26:00,00:03:43,1614,car,car,home,...,43.002341,1,20,1,141.429106,42.993507,POINT (-66477.85615754891 -110508.69842741622),0 days 00:03:43,4.0,True
1,0,14:26:00,14:26:24,05:08:36,19:35:00,00:00:24,132,car,car,eat,...,43.000424,1,20,1,141.429106,42.993507,POINT (-66611.47574067417 -110720.42194019425),0 days 00:00:24,0.0,True
2,0,19:35:00,19:37:41,00:59:19,20:37:00,00:02:41,950,car,car,work,...,42.993354,1,20,1,141.429106,42.993507,POINT (-66857.0550636962 -111503.36158581186),0 days 00:02:41,3.0,True
3,0,20:37:00,20:37:23,NaN,NaN,00:00:23,104,car,car,others,...,42.993507,1,20,1,141.429106,42.993507,POINT (-66936.97317364985 -111485.64818557388),0 days 00:00:23,0.0,True
4,4,12:01:00,12:04:59,01:55:01,14:00:00,00:03:59,1730,car,car,home,...,43.002167,1,20,1,141.429222,42.992758,POINT (-66402.24454050223 -110528.72730580576),0 days 00:03:59,4.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487652,1048548,20:33:00,20:35:51,NaN,NaN,00:02:51,1118,car,car,work,...,43.128232,1,60,1,141.245286,43.128232,POINT (-81746.90187268266 -96356.95418910033),0 days 00:02:51,3.0,True
487653,2097134,06:58:00,07:00:49,06:58:11,13:59:00,00:02:49,1029,car,car,home,...,43.143684,1,60,2,141.316613,43.139511,POINT (-75373.37678761964 -94713.67669700783),0 days 00:02:49,3.0,True
487654,2097134,13:59:00,14:17:43,NaN,NaN,00:18:43,936,walk,walk,retail,...,43.139511,4,60,2,141.316613,43.139511,POINT (-75929.48761091841 -95171.19018994921),0 days 00:18:43,19.0,False
487655,2097148,09:00:00,09:00:08,09:59:52,19:00:00,00:00:08,63,car,car,home,...,43.024592,1,60,2,141.398396,43.024972,POINT (-69354.70877324902 -108008.1761147465),0 days 00:00:08,0.0,True


In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KernelDensity
import folium
from folium.plugins import HeatMap
df = true_df.copy()
# サンプルデータフレームの作成
# 実際のデータフレーム 'df' がある場合は、この行は不要です。
# df = pd.DataFrame({'mesh_o': [53393599, 53393609, 53393588, 53394611, 53394612, 53393598, 53393597, 53394602, 53394603, 53394613]})

def mesh_to_latlon(mesh_code):
    """
    3次メッシュコード（1kmメッシュ）から中心の緯度経度を計算します。
    """
    mesh_code = str(mesh_code)
    if len(mesh_code) != 8:
        raise ValueError("メッシュコードは8桁である必要があります。")

    # 1次メッシュ
    lat1_val = int(mesh_code[0:2])
    lon1_val = int(mesh_code[2:4])
    
    # 2次メッシュ
    lat2_val = int(mesh_code[4])
    lon2_val = int(mesh_code[5])

    # 3次メッシュ
    lat3_val = int(mesh_code[6])
    lon3_val = int(mesh_code[7])

    # 左下の緯度経度を計算 (南西端)
    lat_sw = lat1_val * 2/3 + lat2_val * 1/12 + lat3_val * 1/120
    lon_sw = lon1_val + 100 + lon2_val * 1/8 + lon3_val * 1/80

    # 中心の緯度経度を計算
    # 3次メッシュの大きさは緯度方向に30秒、経度方向に45秒
    lat_center = lat_sw + 30/3600 / 2
    lon_center = lon_sw + 45/3600 / 2
    
    return lat_center, lon_center

try:
    # 'mesh_o'列の各メッシュコードを緯度経度に変換
    df[['lat', 'lon']] = df['mesh_o'].apply(lambda x: pd.Series(mesh_to_latlon(x)))

    # カーネル密度推定の実行
    locations = df[['lat', 'lon']].values

    # bandwidth はデータの分布に応じて調整が必要です。
    # 1kmメッシュの場合、緯度経度の差が約0.01度になるため、それに近い値を設定すると良いでしょう。
    kde = KernelDensity(kernel='gaussian', bandwidth=0.01).fit(locations)

    # 地図の中心を計算
    map_center = [df['lat'].mean(), df['lon'].mean()]

    # Foliumで地図を作成
    m = folium.Map(location=map_center, zoom_start=12)

    # ヒートマップ用のグリッドデータを作成
    lat_min, lat_max = df['lat'].min() - 0.05, df['lat'].max() + 0.05
    lon_min, lon_max = df['lon'].min() - 0.05, df['lon'].max() + 0.05

    # グリッドの数を増やすと滑らかになりますが、計算時間が増加します。
    lat_grid = np.linspace(lat_min, lat_max, 100)
    lon_grid = np.linspace(lon_min, lon_max, 100)
    
    # 経度、緯度の順でグリッドを作成
    xx, yy = np.meshgrid(lon_grid, lat_grid)
    grid_points = np.vstack([yy.ravel(), xx.ravel()]).T

    # 各グリッドポイントでの密度を計算 (score_samplesは対数密度を返す)
    log_density = kde.score_samples(grid_points)
    density = np.exp(log_density)

    # ヒートマップデータを整形
    heat_data = [[point[0], point[1], weight] for point, weight in zip(grid_points, density)]
    
    # ヒートマップレイヤーを地図に追加
    HeatMap(heat_data).add_to(m)

    # 地図をHTMLファイルとして保存
    output_file = 'kernel_density_map.html'
    m.save(output_file)
    
    print(f"✅ カーネル密度推定の結果を '{output_file}' に保存しました。")
    print("このHTMLファイルをブラウザで開いて結果を確認してください。")

except NameError:
    print("⚠️ データフレーム 'df' が定義されていません。")
    print("お手持ちのデータを 'df' という名前のデータフレームに読み込むか、")
    print("プログラム冒頭のサンプルデータフレーム作成部分のコメントアウトを解除して実行してください。")
except KeyError:
    print("⚠️ データフレーム 'df' に 'mesh_o' 列が存在しません。メッシュコードが格納されている列名を確認してください。")
except Exception as e:
    print(f"エラーが発生しました: {e}")